In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request as req
import re

### 爬蟲網站內容

In [2]:
url = 'http://www.atmovies.com.tw/movie/next/' # 開眼電影網 近期上映電影
def crawl(url):
    response = requests.get(url)
    #print(response) # 200表示ok
    print()
    response.encoding = 'utf-8' # 設定編碼使中文字正常顯示
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
    #soup

### 使用正則表達式清洗資料並將其結構化

In [3]:
soup = crawl(url)
movie_list = soup.find('ul', class_='filmListAllX').find_all('li')
#print(movie_list)

site_lst = []
title_lst = []
date_lst = []
runtime_lst  = []

for movie in movie_list:
    # 獲取電影超連結和名稱
    title_div = movie.find('div', class_='filmtitle')
    #print(title_div)
    ## 超連結
    site_str = re.search(r'href="([^"]+)"', str(title_div)) 
    #print(site_str)
    href = site_str.group(1)
    #print(href)
    site_lst.append(href)
    ## 標題
    title_str = re.search(r'<div class="filmtitle"><a href=".*?">(.*?)</a></div>', str(title_div))
    title = title_str.group(1)
    #print(title)
    title_lst.append(title)

    # 獲取上映日期與時長（若有）
    datetime = movie.find('div', class_='runtime')
    #print(datetime)
    #print()
    ## 上映日期
    date_str = re.search(r'\d{4}/\d{2}/\d{2}', str(datetime))
    date = date_str.group()
    #print(date)
    date_lst.append(date)
    ## 片長
    runtime_str = re.search(r'片長：(\d+)分', str(datetime))
    runtime = runtime_str.group(1) if runtime_str else None
    #print(runtime)
    runtime_lst.append(runtime)

# 印出資料並確認長度沒有缺失
list = [title_lst, site_lst, date_lst, runtime_lst]
for data in list:
    print(data)
    print(len(data))
    print()


['特技玩家', '犯罪都市4', '挑戰者', '帝國浩劫：美國內戰', '猩球崛起：王國誕生', '芙莉歐莎：瘋狂麥斯傳奇篇章', '絕地戰警4', '腦筋急轉彎2', '名偵探柯南 100萬美元的五稜星', '神偷奶爸4', '飛月情海', '摩托黑幫', '龍捲風暴', '死侍與金鋼狼', '邊緣禁地', '異形：羅穆路斯', '獵人克萊文', '惡狼特工', '小丑：雙重瘋狂', '荒野機器人', '海洋奇緣2']
21

['/movie/ffen41684562/', '/movie/frkr27811040/', '/movie/fcen16426418/', '/movie/fcen17279496/', '/movie/fken11389872/', '/movie/ffen12037194/', '/movie/fben54919268/', '/movie/fien22022452/', '/movie/fdjp31474158/', '/movie/fden57510222/', '/movie/ffen41896747/', '/movie/fben21454134/', '/movie/ften12584954/', '/movie/fden66263850/', '/movie/fben44978420/', '/movie/faen18412256/', '/movie/fken58790086/', '/movie/fwen14257582/', '/movie/fjen11315808/', '/movie/fwen29623480/', '/movie/fmen13622970/']
21

['2024/04/26', '2024/04/26', '2024/05/01', '2024/05/01', '2024/05/08', '2024/05/22', '2024/06/05', '2024/06/13', '2024/06/28', '2024/07/05', '2024/07/10', '2024/07/12', '2024/07/17', '2024/07/27', '2024/08/09', '2024/08/15', '2024/08/28', '2024/09/28', '2024/10/02', '202

### 把資料存成dataframe與dict

In [4]:
import pandas as pd

data = {'電影': title_lst, '上映日期': date_lst, '時長（分鐘）': runtime_lst, '超連結': site_lst}
# DataFrame
movie_df = pd.DataFrame(data)
display(movie_df)

# 輸出csv
movie_df.to_csv('nextmovie_data.csv', index=False)
# 输出json
movie_df.to_json('nextmovie_data.json', orient='records')

,電影,上映日期,時長（分鐘）,超連結
0,特技玩家,2024/04/26,None,/movie/ffen41684562/
1,犯罪都市4,2024/04/26,109,/movie/frkr27811040/
2,挑戰者,2024/05/01,131,/movie/fcen16426418/
3,帝國浩劫：美國內戰,2024/05/01,None,/movie/fcen17279496/
4,猩球崛起：王國誕生,2024/05/08,None,/movie/fken11389872/
5,芙莉歐莎：瘋狂麥斯傳奇篇章,2024/05/22,None,/movie/ffen12037194/
6,絕地戰警4,2024/06/05,None,/movie/fben54919268/
7,腦筋急轉彎2,2024/06/13,None,/movie/fien22022452/
8,名偵探柯南 100萬美元的五稜星,2024/06/28,None,/movie/fdjp31474158/
9,神偷奶爸4,2024/07/05,None,/movie/fden57510222/


## 用戶查詢功能

In [12]:
def show_des(num):
    
    # 點擊進入特定電影頁面
    url_search = 'http://www.atmovies.com.tw' + movie_df['超連結'][num]
    print(url_search)
    
    soup = crawl(url_search)
    
    print(f"{movie_df['電影'][num]} {movie_df['上映日期'][num]}上映")
    #print()
    
    # 簡介
    abstract_p = re.compile(r'<meta content="(.*?)" property="og:description"/>')
    abstract = abstract_p.search(str(soup))
    print(abstract.group(1))
    print()
    
    # 附加資料
    cast_data = soup.find('div', id='filmCastDataBlock')
    #print(cast_data)
    text_content = cast_data.get_text()
    text_content = '\n'.join(line.strip() for line in text_content.splitlines() if line.strip())
    text_content = text_content.replace('more', '')
    print(text_content)

In [13]:
search_num = int(input('請輸入欲查詢的編號：'))
#display(movie_df.loc[num,])
show_des(search_num)

請輸入欲查詢的編號： 8


http://www.atmovies.com.tw/movie/fdjp31474158/

名偵探柯南 100萬美元的五稜星 2024/06/28上映
柯南劇場版系列第27部作品，由永岡智佳執導，大倉崇裕編劇，故事描述怪盜基德對斧江財團位於北海道函館的收藏庫發出了預告信，然而奇怪的是，向來只愛盜取名貴珠寶的基德，這次目標竟然是一把幕末時代的日本刀。

導演：
永岡智佳
編劇：
大倉崇裕
演員：
高山南
山口勝平
山崎和佳奈
小山力也
堀川亮

IMDb
影片年份：2024
出  品  國：Japan
出　　品：TMS Entertainment
發  行  商：普威爾
語　　言：Japanese
色　　彩：color
音　　效：
